# Introduction to Scikit-learn for Regression Modeling

Using Scikit-learn for regression modeling is very similar to using it for classification modeling.  Your algorithms and evaluation metrics will be the only difference.  Recall, some algorithms for regression modeling include:
- Linear Regression
- K-Nearest Neighbor
- Lasso Regression
<br><br>
There are a few others for regression modeling, however, we will only focus on these in this workshop.

Recall, evaluation metrics tell us how good our model is.  Though there are other types of evaluation metrics, we will only use **R Squared** as a measure of “goodness” of our regression model in this workshop.  **R Squared** is another name for the Coefficient of Determination which is a measure of how well future data are likely to be predicted by the model.  R Squared is generally always between 0 and 1.  The closer this value is to 1, the better the model predicts.  A value less than 1 indicates the model is not useful at all for making predictions. 

For example, if your R Squared is 0.825, this means: 
YOUR MODEL IS 82.5% LIKELY TO PREDICT FUTURE DATA CORRECTLY.

# Using Machine Learning to Predict a Number Value

You are a diamond broker who wants better insight on current and future diamond prices.  You obtain the following data regarding past diamonds sales worldwide:
- price: price in US dollars (326 - 18,823 US dollars)
- carat: weight of the diamond (0.2 - 5.01)
- cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
- color: diamond colour, from J (worst) to D (best)
- clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
- x: length in mm (0 - 10.74)
- y: width in mm (0 - 58.9)
- z: depth in mm (0 - 31.8)
- depth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
- table: width of top of diamond relative to widest point (43--95)

This data is located in the *data* directory.  Let's import this dataset into its own Pandas DataFrame and then print the first 10 rows to the screen.

In [11]:
import pandas as pd
diamonds = pd.read_csv('data/diamonds.csv')
print(diamonds.head(10))

   price  carat        cut color clarity  depth  table     x     y     z
0    326   0.23      Ideal     E     SI2   61.5   55.0  3.95  3.98  2.43
1    326   0.21    Premium     E     SI1   59.8   61.0  3.89  3.84  2.31
2    327   0.23       Good     E     VS1   56.9   65.0  4.05  4.07  2.31
3    334   0.29    Premium     I     VS2   62.4   58.0  4.20  4.23  2.63
4    335   0.31       Good     J     SI2   63.3   58.0  4.34  4.35  2.75
5    336   0.24  Very Good     J    VVS2   62.8   57.0  3.94  3.96  2.48
6    336   0.24  Very Good     I    VVS1   62.3   57.0  3.95  3.98  2.47
7    337   0.26  Very Good     H     SI1   61.9   55.0  4.07  4.11  2.53
8    337   0.22       Fair     E     VS2   65.1   61.0  3.87  3.78  2.49
9    338   0.23  Very Good     H     VS1   59.4   61.0  4.00  4.05  2.39


You decide that the last 9 columns of this dataset (from carat to z) is sufficient for predicting the first column, price.  That is, given this input data, you can create a model to predict the price of a brand-new diamond.  What will future diamond prices be?

### Pre-Processing is Required for Regression Algorithms

Before getting started with the machine learning steps, it has to be mentioned that regression algorithms require **all numerical** inputs.  That is, the values in each column MUST BE NUMBER VALUES.  Notice that the cut, color and clarity columns contain text (or categorical) data.  We have to fix this by transforming the categorical column data to one-hot-encoding format.
<br><br>
For simplicity, I will not discuss this in detail.  You can go [here](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) for a simple tutorial about one-hot-encoding.  Just understand that the next bit of code transforms the categorical column data to numerical data in one-hot-encoding format:

In [12]:
diamonds = pd.concat([diamonds, pd.get_dummies(diamonds['cut'], prefix='cut', drop_first=True)],axis=1)
diamonds = pd.concat([diamonds, pd.get_dummies(diamonds['color'], prefix='color', drop_first=True)],axis=1)
diamonds = pd.concat([diamonds, pd.get_dummies(diamonds['clarity'], prefix='clarity', drop_first=True)],axis=1)
diamonds.drop(['cut','color','clarity'], axis=1, inplace=True)
print(diamonds.head(10))

   price  carat  depth  table     x     y     z  cut_Good  cut_Ideal  \
0    326   0.23   61.5   55.0  3.95  3.98  2.43         0          1   
1    326   0.21   59.8   61.0  3.89  3.84  2.31         0          0   
2    327   0.23   56.9   65.0  4.05  4.07  2.31         1          0   
3    334   0.29   62.4   58.0  4.20  4.23  2.63         0          0   
4    335   0.31   63.3   58.0  4.34  4.35  2.75         1          0   
5    336   0.24   62.8   57.0  3.94  3.96  2.48         0          0   
6    336   0.24   62.3   57.0  3.95  3.98  2.47         0          0   
7    337   0.26   61.9   55.0  4.07  4.11  2.53         0          0   
8    337   0.22   65.1   61.0  3.87  3.78  2.49         0          0   
9    338   0.23   59.4   61.0  4.00  4.05  2.39         0          0   

   cut_Premium      ...       color_H  color_I  color_J  clarity_IF  \
0            0      ...             0        0        0           0   
1            1      ...             0        0        0          

Notice this step created a few more columns.  We will now use the columns from carat to clarity_VVS2 to predict the price column.  Let's begin our machine learning steps.

**STEP 1: Decide on an algorithm. **
<br>
Let’s decide to make our prediction using a Linear Regression model with R Squared as our evaluation metric.  Note that Linear Regression is an appropriate model because we want to predict a number value:  What will future diamond prices be? 
<br><br>
Let's do the imports and then initialize the algorithm:


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()

**STEP 2: Split your data into testing and training subsets. **
<br>
Recall that we do this using **train_test_split( )** on X and y.  Here is our testing and training subsets:

In [14]:
X = diamonds.loc[:, 'carat':'clarity_VVS2']
y = diamonds.loc[:,'price']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In the above code, 20% of the data randomly goes in the testing set and subsequently 80% goes in the training set.

**STEP 3: Use the training data to train your model. **
That is, use X_train and y_train as inputs so that the Linear Regression algorithm can find a pattern in which to make a prediction.  This training is done by passing the training data into the **fit( )** method applied to our **linear_regression( )** algorithm defined in the above code.

In [15]:
linear_regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**STEP 4: Use the testing data to evaluate your model. **
<br>
That is, use X_test to make a prediction about y and then compare y_test to these predicted y values.  Remember our evaluation metric will be R Squared.

In [19]:
y_predicted = linear_regression.predict(X_test)
eval_metric = r2_score(y_pred = y_predicted, y_true = y_test)
print(eval_metric)

0.925055821417


The result tells us that the Linear Regression algorithm is 92.5% likely to predict future diamond prices correctly.

Before moving on to the last step of making a prediction, remember that we want to pick the BEST model to make our prediction.  Here, the Linear Regression model is good, but is it the BEST?  To determine this, we have to re-do some of the above steps for different regression models and then compare their R Squared values.  Whichever one has the highest R Squared wins!
<br><br>
We will do this in part 2 of this notebook, **Scikit-Learn-Regression2.ipynb**.  See you there!